In [1]:
! start .

In [7]:
! code .

In [1]:
import os,git,shutil
import pandas as pd

In [2]:
# 指定されたデータサイズでファイルを分割する
def divide_file(filePath, chunkSize):
    readedDataSize = 0
    i = 0
    fileList = []
    # 対象ファイルを開く
    f = open(filePath, "rb")
    # ファイルを読み終わるまで繰り返す
    contentLength = os.path.getsize(filePath)
    while readedDataSize < contentLength:
        # 読み取り位置をシーク
        f.seek(readedDataSize)
        # 指定されたデータサイズだけ読み込む
        data = f.read(chunkSize)
        # 分割ファイルを保存
        saveFilePath = filePath + "." + str(i)
        with open(saveFilePath, 'wb') as saveFile:
            saveFile.write(data)
        # 読み込んだデータサイズの更新
        readedDataSize = readedDataSize + len(data)
        i = i + 1
        fileList.append(saveFilePath)
    return fileList

# 渡されたファイルリストの順序で１つのファイルに結合する
def join_file(fileList, filePath):
    with open(filePath, 'wb') as saveFile:
        for f in fileList:
            data = open(f, "rb").read()
            saveFile.write(data)
            saveFile.flush()

In [3]:
  class env_util():
    def __init__(self):
        self.u_path = os.environ["userprofile"]
        self.miniconda_path = os.path.join(self.u_path,"miniconda3","pkgs")
        self.freecad_path = os.path.join(self.u_path,"AppData","Roaming","FreeCAD")
        self.fac_path = os.path.join(self.u_path,"000_work","factory_env_miniconda39")
        self.fac_pkg_path = os.path.join(self.fac_path,"pkgs")
        self.repo_fac = git.Repo(self.fac_path)
#         self.repo_freecad = git.Repo(self.freecad_path)
        self.remote_folder_path = os.path.join(self.u_path,"000_work","env_miniconda_39")
        self.remote_pkg_path = os.path.join(self.remote_folder_path,"pkgs")
        self.repo_remote_folder = git.Repo(self.remote_folder_path)

    def get_status_text(self,case):
        if case == 0:
            tex = self.repo_fac.git.status("-s")
        elif case == 1:
            tex = self.repo_remote_folder.git.status("-s")
        elif case == 2:
            tex = self.repo_freecad.git.status("-s")
        return tex
        
    def comfirm_file(self,case):
        tex = self.get_status_text(case)
        if tex.find("\n")  != 0:
            li = tex.split("\n")
            print(li)

        return li

    def get_list(self):
        file_list = []
        file_list_batsu = []
        for root, dirs, files in os.walk(self.miniconda_path):
            for filename in files:
                fname = os.path.join(root, filename)
                file_list.append([fname,os.path.getsize(fname) / 1000000])
            df = pd.DataFrame(file_list)
            df.columns=["path","MB"]
        return df
        
if __name__ == '__main__':
    self = env_util()

# おうち側作業

## pip

In [4]:
import os

In [10]:
os.environ["userprofile"]

'C:\\Users\\M151325'

In [11]:
f_path = os.path.join(os.environ["userprofile"],"000_work","env_miniconda_39","pip_pkgs")
f_path

'C:\\Users\\M151325\\000_work\\env_miniconda_39\\pip_pkgs'

In [12]:
pkg_name = "pymupdf"
# print(f"pip download -d {f_path} --no-binary :all: {pkg_name}")
print(f"pip download -d {f_path} {pkg_name}")

pip download -d C:\Users\M151325\000_work\env_miniconda_39\pip_pkgs pymupdf


In [1]:
! start .

## 初期化

### freecad

未コミットはコミットしておく

In [33]:
li = self.comfirm_file(case=2)

['']


未コミットがあれば、コミットとして0にする

In [9]:
if len(li) != 0:
    self.repo_fac.git.add(r"*")
    li = self.comfirm_file(case=2)

if len(li) != 0:
    self.repo_fac.git.commit("-m","initial commit")

['']


ファイルをコピー

In [10]:
import pandas as pd
li_m = self.comfirm_file(case=2)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df_tg = df[df["code"]=="??"]
li = list(df_tg["file_name"])

['']


In [11]:
li_m

['']

In [14]:
file_list = li
out_list = []
for file in file_list:
    f_path_moto = os.path.join(self.freecad_path,file)
    f_path_copy = os.path.join(self.remote_folder_path,"FreeCAD",file)
    out_list.append([f_path_copy,os.path.getsize(f_path_moto) / 1000000])
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{f_path_moto}")
        pass
df = pd.DataFrame(out_list)
if len(df) != 0:
    df.columns=["path","MB"]

100M以上は分割（factory_env_miniconda39）

In [21]:
files = os.listdir(os.path.join(self.remote_folder_path,"FreeCAD"))
file_list = [os.path.join(self.remote_folder_path,"FreeCAD",f) for f in files ]
file_list = [[f,os.path.getsize(f) / 1000000] for f in file_list ]
df = pd.DataFrame(file_list)
df.columns=["path","MB"]

In [22]:
df_b = df[df["MB"]>99]
df_b

,path,MB


In [16]:
if len(df_b)!=0:
    for target in list(df_b["path"]):
        fileList = divide_file(target, 50*10**6)
        df_a = pd.DataFrame(fileList,columns=["path"])
        df_a.to_csv(os.path.join("sep_file_list_freecad/",os.path.basename(target)+".csv"),index=False)
        os.remove(target)

### env_miniconda_39

pkgsとsep_file_listの中のファイルを削除

In [8]:
shutil.rmtree(self.remote_pkg_path)
os.mkdir(self.remote_pkg_path)
shutil.rmtree(os.path.join(self.remote_folder_path,"sep_file_list"))
os.mkdir(os.path.join(self.remote_folder_path,"sep_file_list"))

### factory_env_miniconda39

未コミットはコミットしておく

In [9]:
li = self.comfirm_file(case=0)

['']


未コミットがあれば、コミットとして0にする

In [10]:
if len(li) != 0:
    self.repo_fac.git.add(r"*")
    li = self.comfirm_file(case=0)

if len(li) != 0:
    self.repo_fac.git.commit("-m","initial commit")

['']


GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git commit -m initial commit
  stdout: 'On branch master
nothing to commit, working tree clean'

## miniconda→factory_env_miniconda39にコピー
同じ状態にします

In [11]:
files = os.listdir(self.miniconda_path)
file_list = [f for f in files if os.path.isfile(os.path.join(self.miniconda_path, f))]
for file in file_list:
    f_path_moto = os.path.join(self.miniconda_path,file)
    f_path_copy = os.path.join(self.fac_pkg_path,file)
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{file}")
        pass

In [12]:
file_list

['.cio-config.py',
 'alabaster-0.7.12-py_0.tar.bz2',
 'altgraph-0.17-pyhd8ed1ab_0.tar.bz2',
 'ansi2html-1.6.0-py39hcbf5309_1.tar.bz2',
 'anyio-3.3.4-py39hcbf5309_0.tar.bz2',
 'appdirs-1.4.4-pyh9f0ad1d_0.tar.bz2',
 'argon2-cffi-20.1.0-py39hb82d6ee_2.tar.bz2',
 'asgiref-3.4.1-pyhd8ed1ab_0.tar.bz2',
 'astroid-2.8.0-py39hcbf5309_0.tar.bz2',
 'async_generator-1.10-py_0.tar.bz2',
 'attrs-21.2.0-pyhd8ed1ab_0.tar.bz2',
 'autopep8-1.5.7-pyhd8ed1ab_0.tar.bz2',
 'babel-2.9.1-pyh44b312d_0.tar.bz2',
 'backcall-0.2.0-pyh9f0ad1d_0.tar.bz2',
 'backports-1.0-py_2.tar.bz2',
 'backports.functools_lru_cache-1.6.4-pyhd8ed1ab_0.tar.bz2',
 'beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2',
 'blas-1.0-mkl.conda',
 'bleach-4.1.0-pyhd8ed1ab_0.tar.bz2',
 'boost-cpp-1.74.0-h5b4e17d_4.tar.bz2',
 'brotli-python-1.0.9-py39h415ef7b_5.tar.bz2',
 'brotlipy-0.7.0-py39h2bbff1b_1003.conda',
 'bzip2-1.0.8-h8ffe710_4.tar.bz2',
 'ca-certificates-2021.10.8-h5b45459_0.tar.bz2',
 'ca-certificates-2021.5.30-h5b45459_0.tar.bz2',
 'ca-

## factory_env_miniconda39→env_miniconda_39にコピー
factory_env_miniconda39の未コミットのファイルをenv_miniconda_39にコピーします

In [ ]:
import pandas as pd
li_m = self.comfirm_file(case=0)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df_tg = df[df["code"]=="??"]
li = list(df_tg["file_name"])

In [ ]:
li_m

In [18]:
file_list = li
out_list = []
for file in file_list:
    f_path_moto = os.path.join(self.fac_path,file)
    f_path_copy = os.path.join(self.remote_folder_path,file)
    out_list.append([f_path_copy,os.path.getsize(f_path_moto) / 1000000])
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{f_path_moto}")
        pass
df = pd.DataFrame(out_list)
if len(df) != 0:
    df.columns=["path","MB"]

## 100M以上は分割（factory_env_miniconda39）

In [19]:
files = os.listdir(self.remote_pkg_path)
file_list = [os.path.join(self.remote_pkg_path,f) for f in files ]
file_list = [[f,os.path.getsize(f) / 1000000] for f in file_list ]
df = pd.DataFrame(file_list)
df.columns=["path","MB"]

In [20]:
df_b = df[df["MB"]>99]
df_b

,path,MB
48,C:\Users\M151325\000_work\env_miniconda_39\pkg...,198.27625


In [21]:
if len(df_b)!=0:
    for target in list(df_b["path"]):
        fileList = divide_file(target, 50*10**6)
        df_a = pd.DataFrame(fileList,columns=["path"])
        df_a.to_csv(os.path.join("sep_file_list/",os.path.basename(target)+".csv"),index=False)
        os.remove(target)

In [22]:
! start .

## githubへpush(factory_env_miniconda39)

In [6]:
li_m = self.comfirm_file(case=1)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df

[' M env.md', ' D pkgs/beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2', ' D pkgs/soupsieve-2.0.1-py_1.tar.bz2', ' D pkgs/xlwings-0.24.9-py39hcbf5309_0.tar.bz2', ' M utility.ipynb', '?? Untitled.ipynb', '?? pkgs/anyio-3.3.4-py39hcbf5309_0.tar.bz2', '?? pkgs/asgiref-3.4.1-pyhd8ed1ab_0.tar.bz2', '?? pkgs/astroid-2.8.0-py39hcbf5309_0.tar.bz2', '?? pkgs/ca-certificates-2021.10.8-h5b45459_0.tar.bz2', '?? pkgs/certifi-2021.10.8-py39hcbf5309_0.tar.bz2', '?? pkgs/cffi-1.14.6-py39h0878f49_1.tar.bz2', '?? pkgs/conda-4.10.3-py39hcbf5309_2.tar.bz2', '?? pkgs/curl-7.79.1-h789b8ee_0.tar.bz2', '?? pkgs/dash-2.0.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/dash-core-components-2.0.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/dash-html-components-2.0.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/dataproperty-0.53.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/decorator-5.1.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/dtale-1.57.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/eigen-3.4.0-h2d74725_0.tar.bz2', '?? pkgs/fastapi-0.70.0-pyhd8ed1ab_0.tar.bz2', '?? pkgs/gitpython-3.1.2

,code,file_name
0,M,env.md
1,D,pkgs/beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2
2,D,pkgs/soupsieve-2.0.1-py_1.tar.bz2
3,D,pkgs/xlwings-0.24.9-py39hcbf5309_0.tar.bz2
4,M,utility.ipynb
...,...,...
83,??,pkgs/vc-14.2-hb210afc_5.tar.bz2
84,??,pkgs/vs2015_runtime-14.29.30037-h902a5da_5.tar...
85,??,pkgs/wheel-0.37.0-pyhd8ed1ab_1.tar.bz2
86,??,pkgs/yaml-0.2.5-he774522_0.tar.bz2


In [7]:
if len(li_m) != 0:
    self.repo_remote_folder.git.add(r"*")
    li_mi= self.comfirm_file(case=1)

['A  Untitled.ipynb', 'M  env.md', 'A  pkgs/anyio-3.3.4-py39hcbf5309_0.tar.bz2', 'A  pkgs/asgiref-3.4.1-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/astroid-2.8.0-py39hcbf5309_0.tar.bz2', 'D  pkgs/beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2', 'A  pkgs/ca-certificates-2021.10.8-h5b45459_0.tar.bz2', 'A  pkgs/certifi-2021.10.8-py39hcbf5309_0.tar.bz2', 'A  pkgs/cffi-1.14.6-py39h0878f49_1.tar.bz2', 'A  pkgs/conda-4.10.3-py39hcbf5309_2.tar.bz2', 'A  pkgs/curl-7.79.1-h789b8ee_0.tar.bz2', 'A  pkgs/dash-2.0.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/dash-core-components-2.0.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/dash-html-components-2.0.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/dataproperty-0.53.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/decorator-5.1.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/dtale-1.57.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/eigen-3.4.0-h2d74725_0.tar.bz2', 'A  pkgs/fastapi-0.70.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/gitpython-3.1.24-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/h11-0.12.0-pyhd8ed1ab_0.tar.bz2', 'A  pkgs/img2pdf-0.4.1-py39hcbf5309_0.tar

In [9]:
li_mi

['A  Untitled.ipynb',
 'M  env.md',
 'A  pkgs/anyio-3.3.4-py39hcbf5309_0.tar.bz2',
 'A  pkgs/asgiref-3.4.1-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/astroid-2.8.0-py39hcbf5309_0.tar.bz2',
 'D  pkgs/beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2',
 'A  pkgs/ca-certificates-2021.10.8-h5b45459_0.tar.bz2',
 'A  pkgs/certifi-2021.10.8-py39hcbf5309_0.tar.bz2',
 'A  pkgs/cffi-1.14.6-py39h0878f49_1.tar.bz2',
 'A  pkgs/conda-4.10.3-py39hcbf5309_2.tar.bz2',
 'A  pkgs/curl-7.79.1-h789b8ee_0.tar.bz2',
 'A  pkgs/dash-2.0.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/dash-core-components-2.0.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/dash-html-components-2.0.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/dataproperty-0.53.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/decorator-5.1.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/dtale-1.57.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/eigen-3.4.0-h2d74725_0.tar.bz2',
 'A  pkgs/fastapi-0.70.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/gitpython-3.1.24-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/h11-0.12.0-pyhd8ed1ab_0.tar.bz2',
 'A  pkgs/img2pdf-0.4

In [8]:
if len(li_mi) != 0:
    self.repo_remote_folder.git.commit("-m","STEP8_iroiror")

push

In [10]:
self.repo_remote_folder.git.push()

''

# 相手側作業

リポジトリをダウンロードして、000_workの下に移動する

https://github.com/nyanko111777/env_miniconda_39/archive/refs/heads/main.zip


## 100M以上を復活

In [4]:
import pandas as pd
import os

file_list =  os.listdir("sep_file_list")
for file in file_list:
    df_temp = pd.read_csv("sep_file_list/"+file)
    fileList = list(list(df_temp["path"]))
    target = fileList[0][:fileList[0].find(".0")]
    join_file(fileList, target)
    for d_file in fileList:
        os.remove(d_file)

['mkl-2020.4-hb70f87d_311.tar.bz2.csv', 'qt-5.12.9-h5909a2a_4.tar.bz2.csv']

## インストール

- conda の場合
  `for /f %i in ('dir /A-d /B *') do conda install %i`



- pip の場合
  `for /f %i in ('dir /A-d /B *') do pip install %i`
